# Out-of-site run times
Version of the out-of-site ablation tests that gets the run-times for the final model and architecture ablation tests.
#### Notes
- Only one fold model is created for each run
- Only 20 runs per test - i.e. timings are for one ensemble of twenty runs.
- No parallel running of tests
- The first test (test0) is the proposed model, so other tests are offset by 1 compared to the full ablation test (e.g. the dropout test is test4 here and test3 in the ablation test)
- By default, training times are output to the `train_stats.csv` file, so these could also be obtained from the ablation tests model directories. This notebook allows testing under more controlled conditions - e.g. if the full runs are run in a shared environment they may not be accurate/consistent due to the server workload. So this is a cut-down version that can run in a small dedicated environment.

In [1]:
import os
import json
import numpy as np
import pandas as pd

import initialise
import common
from model_utils import reshape_data
from modelling_functions import create_models, run_experiment
from architecture_out_of_site import model_params

## Input files
Change these settings as required
- `modis_csv`: The file containing extracted MODIS data for each sample, created by `Extract MODIS Data.ipynb`
- `prism_csv`: The file containing extracted PRISM data for each sample, created by `Extract PRISM Data.ipynb`
- `aux_csv`: The file containing extracted sample labels, DEM, climate zone and other auxiliary data, created by `Extract Auxiliary Data.ipynb`.

In [2]:
modis_csv = os.path.join(common.DATASETS_DIR, 'modis_365days.csv')
prism_csv = os.path.join(common.DATASETS_DIR, 'prism_365days.csv')
aux_csv = os.path.join(common.DATASETS_DIR, 'samples_365days.csv')

## Set up experiment parameters
If the experiment dictionary contains a 'tests' key that is not 'falsy' (False, None, 0, empty list) it is assumed to be a list of tests to run. Each test will run with the specified model parameters. Model parameters not specified will be the same for each test, as set in the main model_params dictionary. A failed run can be restarted by setting the 'restart' key to the test that failed. This test and the remaining tests will then be run.

If 'tests' is 'falsy' then a single test will be run using the parameters in the main model_params dictionary.

Other settings are:
- layerTypes: specifies which layers to include in the model
- Layer parameters should be specified as a list. The first entry in the list will be used for the first layer, etc.
- If the experiment includes changes to the layers, all non-default layer parameters need to be included. The parameters that are kept constant can be specified by including a key for the layer type in the experiment dictionary, and the value set to a dictionary of the constant parameters.

Model_parameters that cannot be changed in tests are:
- \*Filename
- \*Channels
- targetColumn

Example of setting layer parameters:  
```
{'name': 'Filters',  
 'description': 'Test effect of different filter sizes on conv layers',  
 'tests': [{'conv': {'filters': [32, 32, 32]}},  
           {'conv': {'filters': [8, 8, 8]}},  
           {'conv': {'filters': [32, 8, 8]}},  
           {'conv': {'filters': [8, 32, 8]}},  
           {'conv': {'filters': [8, 8, 32]}},  
           {'conv': {'filters': [8, 16, 32]}},  
           {'conv': {'filters': [32, 16, 8]}}],  
 'conv': {'numLayers': 3, 'poolSize': [2, 3, 4]},  
 'restart': 0}
``` 

In [3]:
experiment = {
    'name': 'out-of-site_timings',
    'description': 'Timings for out-of-site architecture changes',
    'layerTypes': ['modisConv', 'prismConv', 'fc'],
    'tests': [
        {},
        {'modisConv': {'numLayers': 3, 'filters': [32, 32, 32], 'poolSize': [2, 3, 4]},
         'prismConv': {'numLayers': 3, 'filters': [32, 32, 32], 'poolSize': [2, 3, 4]}
        },
        {'fc': {'numLayers': 2, 'units': [128, 128]}},
        {'fc': {'numLayers': 1, 'units': [256]}},
        {'dropoutRate': 0.5},
        {'batchSize': 32},
        {'epochs': 100},
    ],
    'testNames': [
        'Proposed model',
        'Conv filters: 32',
        'Dense layers: 2',
        'Dense units: 256',
        'Dropout: 0.5',
        'Batch size: 32',
        'Epochs: 100',
    ],
    'restart': None,
}

# Save and display experiment details
experiment_dir = os.path.join(common.MODELS_DIR, experiment['name'])
restart = experiment.get('restart')
if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)
elif restart is None:
    raise FileExistsError(f'{experiment_dir} exists but restart not requested')
experiment_file = f'experiment{restart}.json' if restart else 'experiment.json'
with open(os.path.join(experiment_dir, experiment_file), 'w') as f:
    json.dump(experiment, f, indent=2)
experiment

{'name': 'out-of-site_timings',
 'description': 'Timings for out-of-site architecture changes',
 'layerTypes': ['modisConv', 'prismConv', 'fc'],
 'tests': [{},
  {'modisConv': {'numLayers': 3,
    'filters': [32, 32, 32],
    'poolSize': [2, 3, 4]},
   'prismConv': {'numLayers': 3,
    'filters': [32, 32, 32],
    'poolSize': [2, 3, 4]}},
  {'fc': {'numLayers': 2, 'units': [128, 128]}},
  {'fc': {'numLayers': 1, 'units': [256]}},
  {'dropoutRate': 0.5},
  {'batchSize': 32},
  {'epochs': 100}],
 'testNames': ['Proposed model',
  'Conv filters: 32',
  'Dense layers: 2',
  'Dense units: 256',
  'Dropout: 0.5',
  'Batch size: 32',
  'Epochs: 100'],
 'restart': None}

## Set up model parameters

### Model parameters settings
To find out more about any parameter, run `model_params.help('<parameter>')`. 

In [4]:
model_params['modelName'] = experiment['name']
model_params['description'] = experiment['description']
model_params['modisFilename'] = modis_csv
model_params['prismFilename'] = prism_csv
model_params['auxFilename'] = aux_csv
model_params['splitMethod'] = 'bySite'
model_params['splitFolds'] = 0
model_params['splitSizes'] = (0.1, 0)
model_params['tempDir'] = common.TEMP_DIR
model_params['modelDir'] = os.path.join(common.MODELS_DIR, model_params['modelName'])
model_params['derivedModels'] = common.DERIVED_MODELS
model_params['seedList'] = [
    441, 780, 328, 718, 184, 372, 346, 363, 701, 358,
    566, 451, 795, 237, 788, 185, 397, 530, 758, 633,
]

if not os.path.exists(model_params['modelDir']):
    os.makedirs(model_params['modelDir'])

model_params

{'modelName': 'out-of-site_timings',
 'description': 'Timings for out-of-site architecture changes',
 'modelClass': 'LfmcTempCnn',
 'modelDir': 'G:\\My Drive\\LFMC Data\\multi_modal_LFMC\\Models\\out-of-site_timings',
 'tempDir': 'C:\\Temp\\LFMC',
 'diagnostics': False,
 'dataSources': ['modis', 'prism', 'aux'],
 'restartRun': None,
 'derivedModels': {'merge10': {'type': 'merge', 'models': 10}},
 'saveModels': False,
 'saveTrain': None,
 'saveValidation': True,
 'plotModel': True,
 'randomSeed': 1234,
 'modelSeed': 1234,
 'modelRuns': 3,
 'resplit': False,
 'seedList': [441,
  780,
  328,
  718,
  184,
  372,
  346,
  363,
  701,
  358,
  566,
  451,
  795,
  237,
  788,
  185,
  397,
  530,
  758,
  633],
 'maxWorkers': 1,
 'deterministic': False,
 'gpuDevice': 0,
 'gpuMemory': 0,
 'modisFilename': 'G:\\My Drive\\LFMC Data\\multi_modal_LFMC\\Datasets\\modis_365days.csv',
 'modisChannels': 7,
 'modisNormalise': {'method': 'minMax', 'percentiles': 2},
 'prismFilename': 'G:\\My Drive\\LF

## Prepare the data

In [5]:
modis_data = pd.read_csv(model_params['modisFilename'], index_col=0)
x_modis = reshape_data(np.array(modis_data), model_params['modisChannels'])
print(f'Modis shape: {x_modis.shape}')

prism_data = pd.read_csv(model_params['prismFilename'], index_col=0)
x_prism = reshape_data(np.array(prism_data), model_params['prismChannels'])
print(f'Prism shape: {x_prism.shape}')

aux_data = pd.read_csv(model_params['auxFilename'], index_col=0)
y = aux_data[model_params['targetColumn']]

Modis shape: (5000, 365, 7)
Prism shape: (5000, 365, 7)


## Build and run the models
Builds and trains the LFMC models. After training each model, several derived models are created and evaluated. The full list of models is:
- `base` - The fully trained model
- `best` - A model using the checkpoint with the best training loss
- `merge10` - A model created by merging the last 10 checkpoints. The checkpoints are merged by averaging the corresponding weights from each model.
- `ensemble10` - An ensembled model of the last 10 checkpoints. This model averages the predictions made by each model in the ensemble to make the final prediction.
- `merge_best10` - Similar to the merge10 model, but uses the 10 checkpoints with the lowest training/validation losses.

All models, predictions, evaluation statistics, and plots of test results are saved to `model_dir`, with each test and run saved to a separate sub-directory. For each model created, predictions and evaluation statistics are also returned as attributes of the `model` object. These are stored as nested lists, the structure for a full experiment is:
- Tests (omitted if not an experiment)
  - Runs (omitted for a single run)
    - Folds (for k-fold splitting)

In [6]:
def is_experiment():
    try:
        return bool(experiment['tests'])
    except:
        return False

In [7]:
X = {'modis': x_modis, 'prism': x_prism}
if is_experiment():
    ex_models = run_experiment(experiment, model_params, aux_data, X, y)
else:
    print('Running a single test')
    with open(os.path.join(model_params['modelDir'], 'model_params.json'), 'w') as f:
        model_params.save(f)
    models = create_models(model_params, aux_data, X, y)

Experiment out-of-site_timings - Timings for out-of-site architecture changes

Test Proposed model - {}

Auxiliary columns: ['Elevation', 'Slope', 'Aspect_sin', 'Aspect_cos', 'Long_sin', 'Long_cos', 'Lat_norm']
modis shape: (5000, 365, 7)
prism shape: (5000, 365, 7)
aux shape: (5000, 31)
Split by Site, stratify by Land Cover

----------------------------------------------------------------------

Test Conv filters: 32 - {'modisConv': {'numLayers': 3, 'filters': [32, 32, 32], 'poolSize': [2, 3, 4]}, 'prismConv': {'numLayers': 3, 'filters': [32, 32, 32], 'poolSize': [2, 3, 4]}}

Auxiliary columns: ['Elevation', 'Slope', 'Aspect_sin', 'Aspect_cos', 'Long_sin', 'Long_cos', 'Lat_norm']
modis shape: (5000, 365, 7)
prism shape: (5000, 365, 7)
aux shape: (5000, 31)
Split by Site, stratify by Land Cover

----------------------------------------------------------------------

Test Dense layers: 2 - {'fc': {'numLayers': 2, 'units': [128, 128]}}

Auxiliary columns: ['Elevation', 'Slope', 'Aspect_s

## Display the training times
Time are Tensorflow/Keras model training time and excludes data preparation time.

In [13]:
train_times = []
for test in ex_models:
    run_time = 0
    for run in test:
        df = pd.read_csv(os.path.join(run.model_dir, 'train_stats.csv'))
        run_time += df.trainTime[0]
    weights = df.trainableWeights[0]
    train_times.append([run_time/60, run_time/60/len(test), weights])
pd.DataFrame(train_times, index=experiment['testNames'],
             columns=['ensemble_time', 'single_time', 'num_params']).round(2)

,ensemble_time,single_time,num_params
Proposed model,0.29,0.10,37185
Conv filters: 32,0.50,0.17,150657
Dense layers: 2,0.30,0.10,53953
Dense units: 256,0.29,0.10,72385
Dropout: 0.5,0.31,0.10,37185
Batch size: 32,0.59,0.20,37185
Epochs: 100,1.81,0.60,37185
